# Topic Modeling

Christoph & Nadia

## LDA (Latent Dirichlet Allocation)

## Imports von hauptsächlich Gensim & SpaCy

In [15]:
from pprint import pprint
import gensim
from gensim import corpora, models, similarities
from gensim.models import LdaModel, LsiModel
from spacy_preprocessing.preprocess import Preprocess
from tabulate import tabulate

import numpy as np
import pickle
import pyLDAvis
import pyLDAvis.gensim

import os
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import widgets
from IPython.display import display, clear_output

from spacy_preprocessing.preprocess import Preprocess
import datetime

load_saved = True

## Zusätzliche Preprocessing Funktionen

In [16]:
def is_blacklisted(word):
    return word in [
        'polizei', 'polizist', 'beamter', 'nr.', 'berlin', 'uhr', 'polizeimeldung',
        'nicht', 'jahr', 'jährige', 'jährig', 'jähriger', 'polizeiliche', 'polizeilich', '2015', '2016',
        '2014', '2017', '2018', 'polizeibeamter', '-', 'u.a.', 'z.b.', 'der', 'die', 'das', 'dem', 'den', 'diese', 'dieser',
        'diesen', 'diesem', 'um', 'für', 'eine', 'ein', 'einer', 'einen', 'einem', 'anderer', 'andere', 'anderen', 'anders'
    ]

def is_empty(word):
    return word.strip() == ''

def can_parse_date(word):
    try:
        datetime.datetime.strptime(word, '%d.%m.%Y')
        return True
    except ValueError:
        return False

def can_parse_num_int(word):
    try:
        int(word)
        return True
    except ValueError:
        return False
    
def can_parse_num_float(word):
    try:
        float(word)
        return True
    except ValueError:
        return False
    
def preprocess_after(doc):
    return [
        word 
        for word in doc
        if not is_empty(word)
        and not is_blacklisted(word)
        and not can_parse_date(word)
        and not can_parse_num_int(word)
        and not can_parse_num_float(word)
    ]

## Daten aus pickle Datei laden und Preprocessing durchführen

In [29]:
items = pickle.load(open('./data/items.pkl', 'rb'))
texts = [report['text'] for report in items]
titles = [report['title'] for report in items]
ids = [report['_id'] for report in items]
data = [report['text_pre_processed_v1'] for report in items]

In [30]:
clean_data = [preprocess_after(doc) for doc in data]

## Dictionary und Bag of Words aus jedem Dokument erstellen

In [31]:
if load_saved:
    id2word = corpora.Dictionary.load('./data/id2word')
    corpus = pickle.load(open('./data/corpus.pkl', 'rb'))
else:
    id2word = corpora.Dictionary(clean_data)
    corpus = [id2word.doc2bow(doc) for doc in clean_data]
    
    id2word.save('./data/id2word')
    pickle.dump(corpus, open('./data/corpus.pkl', 'wb'))

## LDA/LSI Modell

In [32]:
if load_saved:
    lda_model = LdaModel.load('./data/lda_model')
    lsi_model = LsiModel.load('./data/lsi_model')
else: 
    lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=25, passes=50, alpha='auto', eta='auto')
    lsi_model = LsiModel(corpus=corpus, id2word=id2word, num_topics=25, power_iters=50)
    lda_model.save('./data/lda_model')
    lsi_model.save('./data/lsi_model')

In [33]:
pprint(lda_model.print_topics())

[(24,
  '0.018*"dienst" + 0.014*"gesundbrunnen" + 0.012*"raubkommissariate" + '
  '0.010*"insgesamt" + 0.010*"gehoben" + 0.010*"getränk" + 0.008*"zeit" + '
  '0.008*"mitarbeiter" + 0.008*"mitte" + 0.007*"genannt"'),
 (14,
  '0.038*"fahrzeug" + 0.024*"versuchen" + 0.023*"audi" + 0.019*"wasser" + '
  '0.018*"stellen" + 0.016*"schwer" + 0.016*"flucht" + 0.015*"stehlen" + '
  '0.015*"fahrer" + 0.014*"versuch"'),
 (6,
  '0.022*"straße" + 0.010*"berliner" + 0.009*"finden" + 0.009*"platz" + '
  '0.009*"folgend" + 0.008*"tiergarten" + 0.008*"sicherheit" + 0.008*"bereich" '
  '+ 0.008*"informieren" + 0.008*"thema"'),
 (16,
  '0.016*"insgesamt" + 0.011*"rahmen" + 0.008*"droge" + 0.007*"pferd" + '
  '0.007*"verstoß" + 0.007*"handel" + 0.007*"kontrollieren" + '
  '0.006*"flughafen" + 0.006*"bezirksübergreifend" + 0.006*"haschisch"'),
 (15,
  '0.061*"steglitz" + 0.053*"zehlendorf" + 0.034*"km/h" + 0.023*"höhe" + '
  '0.018*"euro" + 0.017*"bußgeld" + 0.016*"fahrverbot" + 0.016*"unterwegs" + '
  '0.0

## Topics visualisieren

In [34]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.162944  0.200377       1        1  15.172674
11     0.200768 -0.045956       2        1  14.380863
13     0.118990 -0.166969       3        1   8.395141
18     0.164839 -0.053787       4        1   8.391598
2      0.123572  0.064180       5        1   5.582594
4      0.018483 -0.173209       6        1   5.062374
22     0.010931  0.121761       7        1   4.836340
20    -0.042544 -0.060539       8        1   4.132868
1      0.092758  0.238137       9        1   3.936636
10     0.079537 -0.174245      10        1   3.417765
21    -0.000192 -0.161767      11        1   3.054197
17     0.071574  0.151225      12        1   2.869653
12     0.067427 -0.046363      13        1   2.307963
6     -0.230476 -0.006360      14        1   2.052934
16    -0.269894  0.014775      15        1   2.052689
5     -0.024660  0.058259      16        1   1.882969
0      0.032280 -0.109424      17        1   1.765434
23     0.081220 -0.003971      18        1   1.644750
3     -0.013328 -0.057372      19        1   1.582245
8     -0.093321  0.076762      20        1   1.549687
7     -0.111065  0.027214      21        1   1.380338
15    -0.132156  0.118289      22        1   1.331531
9     -0.097817 -0.067048      23        1   1.160240
24    -0.201857 -0.038552      24        1   1.144416
14    -0.008011  0.094582      25        1   0.912102, topic_info=      Category         Freq               Term        Total  loglift  logprob
term                                                                         
25     Default  1333.000000     charlottenburg  1333.000000  30.0000  30.0000
703    Default  1519.000000          kreuzberg  1519.000000  29.0000  29.0000
9      Default  3848.000000        krankenhaus  3848.000000  28.0000  28.0000
159    Default  9195.000000               mann  9195.000000  27.0000  27.0000
17     Default  3040.000000         unbekannte  3040.000000  26.0000  26.0000
1439   Default  1243.000000     friedrichshain  1243.000000  25.0000  25.0000
141    Default  1010.000000             pankow  1010.000000  24.0000  24.0000
207    Default  3311.000000             schwer  3311.000000  23.0000  23.0000
631    Default  2072.000000           fahrzeug  2072.000000  22.0000  22.0000
53     Default   943.000000        wilmersdorf   943.000000  21.0000  21.0000
305    Default  4200.000000          verletzen  4200.000000  20.0000  20.0000
70     Default  2169.000000          feuerwehr  2169.000000  19.0000  19.0000
209    Default  2501.000000          stationär  2501.000000  18.0000  18.0000
68     Default  7443.000000         ermittlung  7443.000000  17.0000  17.0000
136    Default  3220.000000               frau  3220.000000  16.0000  16.0000
81     Default  1936.000000  landeskriminalamt  1936.000000  15.0000  15.0000
222    Default  1645.000000            wohnung  1645.000000  14.0000  14.0000
251    Default  1604.000000             fahrer  1604.000000  13.0000  13.0000
33     Default  1553.000000               geld  1553.000000  12.0000  12.0000
173    Default  2508.000000            bringen  2508.000000  11.0000  11.0000
49     Default  5148.000000             straße  5148.000000  10.0000  10.0000
57     Default  3287.000000         alarmieren  3287.000000   9.0000   9.0000
229    Default  2233.000000               auto  2233.000000   8.0000   8.0000
7      Default  3114.000000           flüchten  3114.000000   7.0000   7.0000
51     Default  1982.000000              täter  1982.000000   6.0000   6.0000
2      Default  3195.000000         behandlung  3195.000000   5.0000   5.0000
347    Default  1936.000000         mutmaßlich  1936.000000   4.0000   4.0000
304    Default  1672.000000     verkehrsunfall  1672.000000   3.0000   3.0000
751    Default  1137.000000        angestellte  1137.000000   2.0000   2.0000
758    Default  1128.000000             räuber  1128.000000   1.0000   1.0000
...

## MatrixSimilarity erstellen

In [35]:
if load_saved:
    index = similarities.MatrixSimilarity.load('./data/index')
else:
    index = similarities.MatrixSimilarity(lda_model[corpus])
    index.save('./data/index')

## Die Suche

In [37]:
use_lsi = True

html_widget = widgets.HTML(
    value = ''
)

text_widget = widgets.Text()


def search(sender):  
    preprocess = Preprocess(sender.value)
    search_term_preprocessed = preprocess_after(preprocess.preprocess(sentence_split=False, with_pos=False))
    search_term_bow = id2word.doc2bow(search_term_preprocessed)
    
    if use_lsi:
        search_term_vec = lsi_model[search_term_bow]
    else:
        search_term_vec = lda_model[search_term_bow]
    
    
    sims = index[search_term_vec]
    similarity_list = list(zip(range(len(sims)), sims, texts, titles, ids))
    similarity_list.sort(key=lambda x: x[1], reverse=True)
    
    results = [f'<h3>{result[3]}</h3><em>Id: {result[4]} | Similarity Score: {round(result[1] * 100, 2)} %</em><br><p style="width: 80%;">{result[2]}</p>' for result in similarity_list[:10]]
    output = '<hr><br>'.join(results)
    
    html_widget.value = output

text_widget.on_submit(search)    
    
display(text_widget)
display(html_widget)    


Text(value='')

HTML(value='')